# Ejemplo de Uso de la Herramienta AutoGen


## Importar los paquetes necesarios


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Creando el Cliente

En este ejemplo, utilizaremos [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) para acceder al LLM.

El `model` está definido como `gpt-4o-mini`. Intenta cambiar el modelo a otro disponible en el mercado de GitHub Models para ver los diferentes resultados.

Como prueba rápida, simplemente ejecutaremos un mensaje sencillo: `¿Cuál es la capital de Francia?`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Definiendo las Funciones

En este ejemplo, le daremos al agente acceso a una herramienta que es una función con una lista de destinos de vacaciones disponibles y su disponibilidad.

Puedes imaginar que este sería un escenario donde un agente de viajes podría tener acceso a una base de datos de viajes, por ejemplo.

A medida que revises este ejemplo, siéntete libre de intentar definir nuevas funciones y herramientas que el agente pueda utilizar.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Definiendo la Herramienta Función
Para que el agente utilice `vacation_destinations` como un `FunctionTool`, necesitamos definirlo como tal.

También proporcionaremos una descripción de la herramienta, lo cual es útil para que el agente identifique para qué se utiliza esa herramienta en relación con la tarea que el usuario ha solicitado.


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Definiendo el agente

Ahora podemos crear el agente en el siguiente código. Definimos el `system_message` para darle al agente instrucciones sobre cómo ayudar a los usuarios a encontrar destinos de vacaciones.

También configuramos el parámetro `reflect_on_tool_use` en true. Esto permite que el LLM tome la respuesta de la llamada a la herramienta y envíe la respuesta utilizando lenguaje natural.

Puedes configurar el parámetro en false para ver la diferencia.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Ejecutar el Agente

Ahora podemos ejecutar el agente con el mensaje inicial del usuario pidiendo hacer un viaje a Tokio.

Puedes cambiar este destino de ciudad para ver cómo responde el agente a la disponibilidad de la ciudad.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automatizadas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
